In [ ]:
"""
Created on Wed Feb 01 16:21 2023

Check the input masks (grounding line, ice front, ice shelves, ...)

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import matplotlib as mpl
from tqdm.notebook import tqdm
import basal_melt_param.plume_functions as pf
from scipy.interpolate import griddata
import basal_melt_param.create_isf_mask_functions as isfmf


In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'bi646'
map_lim = [-3000000,3000000]


In [ ]:
outputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_data2='/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_OPM016_ANT_STEREO/'

outputpath_boxes = '/bettik/burgardc/DATA/NN_PARAM/interim/BOXES/SMITH_'+nemo_run+'/'
inputpath_raw = '/bettik/burgardc/DATA/NN_PARAM/raw/'

In [ ]:
file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy54.nc')

file_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_mask_cut = uf.cut_domain_stereo(file_mask, map_lim, map_lim)

file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_2000.nc')
file_TS_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim)

In [ ]:
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc')
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
def def_isf_mask(arr_def_ismask, file_msk, lon, lat, add_fac, FRIS_one=True, mouginot_basins=False):
    
    """
    Define a mask for the individual ice shelves. 
    
    This function defines a mask for the individual ice shelves. I think it works for both stereographic and latlon grids but I have not tried the latter.
    
    Parameters
    ----------
    arr_def_ismask : np.array
        Array containing minlon,maxlon,minlat,maxlat,is_nb or a dataarray containing masks of the basins.
    file_msk : xr.DataArray
        Mask separating ocean (0), ice shelves (between 0 and 2, excluding 0 and 2), grounded ice (2) 
    lon : xr.DataArray
        Longitude (depends on x,y for stereographic)
    lat : xr.DataArray
        Latitude (depends on x,y for stereographic)

    Returns
    -------
    new_mask : xr.DataArray
        Array showing the coverage of each ice shelf with the respective ID, open ocean is 1, land is 0
    """    
    
    if mouginot_basins:
        
        isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))
        isf_mask_all = arr_def_ismask.where(np.isfinite(isf_only_mask))
        
        # fine-tuning corrections of ice shelves that belong together
        isf_mask_merged = isf_mask_all.where(
            isf_mask_all != 58, 57).where(
            isf_mask_all != 104, 103).where(
            isf_mask_all != 82, 83).where(
            isf_mask_all != 157, 89).where(
            isf_mask_all != 156, 89).where(
            isf_mask_all != 149, 89).where(
            isf_mask_all != 147, 89).where(
            isf_mask_all != 148, 84).where(
            isf_mask_all != 127, 84).where(
            isf_mask_all != 146, 85).where(
            isf_mask_all != 153, 72)
        
        #isf_mask_withRossmerged = isf_mask_all.where(isf_mask_all != 58, 57)
        #isf_mask_withRossFRISmerged = isf_mask_withRossmerged.where(isf_mask_withRossmerged != 104, 103)
        #isf_mask_withRossFRISStangemerged = isf_mask_withRossFRISmerged.where(isf_mask_withRossFRISmerged != 82, 83)
        #isf_mask_merged1 = isf_mask_withRossFRISStangemerged.where(isf_mask_withRossFRISStangemerged != 157, 89)
        #isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 156, 89)
        #isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 149, 89)
        #isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 147, 89)
        #isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 148, 84)
        #isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 127, 84)
        #isf_mask_merged1 = isf_mask_merged2.where(isf_mask_merged2 != 146, 85)
        #isf_mask_merged2 = isf_mask_merged1.where(isf_mask_merged1 != 153, 72)
        #isf_mask_merged = isf_mask_merged2
        
        # seeding through some ice shelves that are scattered to keep the main part
        isf_mask = isf_mask_merged.copy()

        for kisf in [83,89,84,85,73,72]:
            
            isf_domain = isf_mask.where(isf_mask == kisf, drop=True)

            meshx_gnd, meshy_gnd = np.meshgrid(isf_domain.x,isf_domain.y)
            meshx_gnd_da = isf_domain.copy(data=np.broadcast_to(meshx_gnd, isf_domain.shape))
            meshy_gnd_da = isf_domain.copy(data=np.broadcast_to(meshy_gnd, isf_domain.shape))

            max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
            half_range = round(max_len_xy/2)

            dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
            dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

            if kisf == 83: # Stange
                seed_lon = -76.722
                seed_lat = -73.200
            elif kisf == 89: # Wilkins
                seed_lon = -73.001
                seed_lat = -70.721
            elif kisf == 84: # George VI
                seed_lon = -70.041
                seed_lat = -72.764
            elif kisf == 85: # Bach
                seed_lon = -72.505
                seed_lat = -72.151
            elif kisf == 73: #Thwaites
                seed_lon = -106.744
                seed_lat = -75.1
            elif kisf == 72: #Crosson
                seed_lon = -110.204
                seed_lat = -75.141

            center_x,center_y = uf.change_coord_latlon_to_stereo(seed_lon,seed_lat)

            seed_mask = isf_only_mask.where(meshx_gnd_da, drop=True)
            seed_mask = seed_mask.where(seed_mask > 0) * 0 + 1
            seed_mask = seed_mask.where(np.isfinite(seed_mask),0)
            
            mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dx + center_x,center_x + 1*dx]) # assuming that the South Pole is in the center of the projection
                                  & uf.in_range(meshy_gnd_da, [-1*dy + center_y,center_y +1*dy]))), 5)

            # filter that checks the point around
            weights_filter = np.zeros((3,3))
            weights_filter[0,1] = 1
            weights_filter[1,0] = 1
            weights_filter[1,2] = 1
            weights_filter[2,1] = 1

            weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

            iter_mask = mask_core.copy()
            for n in range(half_range+2+add_fac):
                corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
                iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

            iter_mask = iter_mask.where(iter_mask == 5)
            iter_mask = iter_mask * 0 + kisf

            kisf_reg_grid = iter_mask.reindex_like(isf_mask) 
            isf_mask = isf_mask.where(isf_mask != kisf, kisf_reg_grid)    
        
        isf_mask = isf_mask + 1 # to allow land to be 1
    
    else:
    
        arr_def_general = arr_def_ismask[arr_def_ismask[:, 3] == -50]
        arr_def_detail = arr_def_ismask[arr_def_ismask[:, 3] != -50]

        isf_yes = (file_msk > 0) & (file_msk < 2)
        isf_mask = file_msk.copy()
        # is_mask0.plot()
        for i, mm in enumerate(arr_def_general):
            #print('general ' + str(i))
            isf_mask = isf_mask.where(~(uf.in_range(lon, mm[0:2]) & uf.in_range(lat, mm[2:4])), int(mm[4]))
        for i, mm in enumerate(arr_def_detail):
            #print('detail ' + str(i))
            isf_mask = isf_mask.where(~(uf.in_range(lon, mm[0:2]) & uf.in_range(lat, mm[2:4])), int(mm[4]))
        isf_mask = isf_mask.where(isf_yes)

        if FRIS_one:
            isf_mask = isf_mask.where(isf_mask != 21, 11) # Filchner (21) and Ronne (11) are combined

    new_mask = isf_mask.where(file_msk != 0, 1).where(file_msk != 2, 0)
    
    return new_mask
    #return isf_mask

In [ ]:
seed_mask = isf_only_mask.where(meshx_gnd_da, drop=True)

In [ ]:
seed_mask = seed_mask.where(seed_mask > 0) * 0 + 1
seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

In [ ]:
seed_mask.plot()

In [ ]:
isf_mask_all = summary_mask_basins['ID_isf'].where(np.isfinite(isf_only_mask))

In [ ]:
new_mask = def_isf_mask(summary_mask_basins['ID_isf'], file_msk.isel(time=0), file_isf.longitude, file_isf.latitude, 100, FRIS_one=True, mouginot_basins=True)

In [ ]:
new_mask.plot(vmin=71,vmax=74)

In [ ]:
isf_mask_all.where(isf_mask_all != 153, 72).plot()

In [ ]:
isf_mask_all.plot(vmin=151,vmax=154)

In [ ]:
summary_mask_basins['Nisf'] = summary_mask_basins['Nisf']+1

In [ ]:
summary_mask_basins['Nisf']

In [ ]:
file_conc_cut['isfdraft_conc'].isel(time=0).where(new_mask<2).plot()

In [ ]:
summary_mask_basins = xr.open_dataset('/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc')
file_msk = file_mask_cut['ls_mask012']

In [ ]:
file_msk = file_mask_cut['ls_mask012']
isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))
isf_mask_all = summary_mask_basins['ID_isf'].where(np.isfinite(isf_only_mask))

In [ ]:
new_ground_mask = isfmf.def_ground_mask(file_msk.isel(time=0), 40, 120)

In [ ]:
new_ground_mask2 = isfmf.def_ground_mask(file_msk, 40, 120)

In [ ]:
isf_mask_tt = isf_mask_all.isel(time=0)

In [ ]:
plt.contour(new_ground_mask.x,new_ground_mask.y,new_ground_mask,levels=[0,1],linewidths=0.5,colors='black',zorder=10)
file_conc_cut['isfdraft_conc'].isel(time=0).where(file_conc_cut['isfdraft_conc'].isel(time=0)>0).plot()

In [ ]:
plt.contour(new_ground_mask2.x,new_ground_mask2.y,new_ground_mask2,levels=[0,1],linewidths=0.5,colors='black',zorder=10)
file_conc_cut['isfdraft_conc'].where(file_conc_cut['isfdraft_conc']>0).plot()

MERGING RELEVANT CONTENT

In [ ]:
isf_mask_tt_withRossmerged = isf_mask_tt.where(isf_mask_tt != 58, 57)
isf_mask_tt_withRossFRISmerged = isf_mask_tt_withRossmerged.where(isf_mask_tt_withRossmerged != 104, 103)
isf_mask_tt_withRossFRISStangemerged = isf_mask_tt_withRossFRISmerged.where(isf_mask_tt_withRossFRISmerged != 82, 83)
isf_mask_tt_merged1 = isf_mask_tt_withRossFRISStangemerged.where(isf_mask_tt_withRossFRISStangemerged != 157, 89)
isf_mask_tt_merged2 = isf_mask_tt_merged1.where(isf_mask_tt_merged1 != 156, 89)
isf_mask_tt_merged1 = isf_mask_tt_merged2.where(isf_mask_tt_merged2 != 149, 89)
isf_mask_tt_merged2 = isf_mask_tt_merged1.where(isf_mask_tt_merged1 != 147, 89)
isf_mask_tt_merged1 = isf_mask_tt_merged2.where(isf_mask_tt_merged2 != 148, 84)
isf_mask_tt_merged2 = isf_mask_tt_merged1.where(isf_mask_tt_merged1 != 127, 84)
isf_mask_tt_merged1 = isf_mask_tt_merged2.where(isf_mask_tt_merged2 != 146, 85)
isf_mask_tt_merged1 = isf_mask_tt_merged2.where(isf_mask_tt_merged2 != 153, 72)

In [ ]:
summary_mask_basins['name_isf'].loc[{'Nisf': 57}] = 'Ross'
summary_mask_basins['name_isf'].loc[{'Nisf': 58}] = np.nan
summary_mask_basins['name_isf'].loc[{'Nisf': 103}] = 'Filchner-Ronne'
summary_mask_basins['name_isf'].loc[{'Nisf': 104}] = np.nan
summary_mask_basins['name_isf'] = summary_mask_basins['name_isf'].dropna('Nisf')

In [ ]:
plt.figure()
file_conc_cut['isfdraft_conc'].isel(time=0).where(wilkins, drop=True).plot()

In [ ]:
isf_mask_tt_merged2.plot()

In [ ]:
kisf = 72 
plt.figure()
plt.contour(new_ground_mask.x,new_ground_mask.y,new_ground_mask,levels=[0,1],linewidths=0.5,colors='black',zorder=10)
#isf_mask_tt.where(isf_mask_tt==kisf).plot()
isf_mask.where(isf_mask==kisf).plot()
plt.title(summary_mask_basins['name_isf'].sel(Nisf=kisf).values)

# 12: a few ice shelves in the Northeastern Peninsula
# 15: a few ice shelves south of Wilma
# 18: no idea, but must be South of Publications
# 80: could be renamed Ferrigno

# 72: Crosson - problem
# 74: Pine Island, could be larger - but probably ok like this also
# 83: Stange has a small appendix on an island where it should not 
# 89: Wilkins same as above
# 84: George VI also missing a tip
# 85: Bach missing the tip

# MERGE 
# 82 and 83
# 57 and 58
# 103 and 104
# 89, 157, 156, 149, 147, 148, 128
# 

In [ ]:
file_msk = file_msk.isel(time=0)


In [ ]:
#### THIS WORKS =D 
add_fac = 100
        
isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))
isf_mask_all = summary_mask_basins['ID_isf'].where(np.isfinite(isf_only_mask))

# fine-tuning corrections of ice shelves that belong together
isf_mask_merged = isf_mask_all.where(
    isf_mask_all != 58, 57).where(
    isf_mask_all != 104, 103).where(
    isf_mask_all != 82, 83).where(
    isf_mask_all != 157, 89).where(
    isf_mask_all != 156, 89).where(
    isf_mask_all != 149, 89).where(
    isf_mask_all != 147, 89).where(
    isf_mask_all != 148, 84).where(
    isf_mask_all != 127, 84).where(
    isf_mask_all != 146, 85).where(
    isf_mask_all != 153, 72)


# seeding through some ice shelves that are scattered to keep the main part
isf_mask = isf_mask_merged.copy()

for kisf in [83,89,84,85,73,72]:
    

    isf_domain = isf_mask.where(isf_mask == kisf, drop=True)

    meshx_gnd, meshy_gnd = np.meshgrid(isf_domain.x,isf_domain.y)
    meshx_gnd_da = isf_domain.copy(data=np.broadcast_to(meshx_gnd, isf_domain.shape))
    meshy_gnd_da = isf_domain.copy(data=np.broadcast_to(meshy_gnd, isf_domain.shape))

    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)

    dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
    dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

    if kisf == 83: # Stange
        seed_lon = -76.722
        seed_lat = -73.200
    elif kisf == 89: # Wilkins
        seed_lon = -73.001
        seed_lat = -70.721
    elif kisf == 84: # George VI
        seed_lon = -70.041
        seed_lat = -72.764
    elif kisf == 85: # Bach
        seed_lon = -72.505
        seed_lat = -72.151
    elif kisf == 73: #Thwaites
        seed_lon = -106.744
        seed_lat = -75.1
    elif kisf == 72: #Crosson
        seed_lon = -110.204
        seed_lat = -75.141

    center_x,center_y = uf.change_coord_latlon_to_stereo(seed_lon,seed_lat)

    seed_mask = isf_mask.where(isf_domain, drop=True)
    seed_mask = seed_mask.where(seed_mask > 0) * 0 + 1
    seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dx + center_x,center_x + 1*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dy + center_y,center_y +1*dy]))), 5)

    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1

    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

    iter_mask = mask_core.copy()
    for n in range(half_range+2+add_fac):
        corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
        iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

    iter_mask = iter_mask.where(iter_mask == 5)
    iter_mask = iter_mask * 0 + kisf

    kisf_reg_grid = iter_mask.reindex_like(isf_mask) 
    isf_mask = isf_mask.where(isf_mask != kisf, kisf_reg_grid)   

In [ ]:
plt.figure()
iter_mask.plot()

In [ ]:
isf_mask.plot(vmin=70,vmax=74)

In [ ]:
file_conc_cut['isfdraft_conc'].isel(time=0).where(isf_mask<1).plot()

In [ ]:
isf_mask_tt_merged2.where(meshx_gnd_da).max()

In [ ]:
plt.contour(new_ground_mask.x,new_ground_mask.y,new_ground_mask,levels=[0,1],linewidths=0.5,colors='black',zorder=10)
kisf_reg_grid.plot()

In [ ]:
isf_mask_tt_withRossFRISStangemerged.plot()
plt.scatter(center_x,center_y,c=None,edgecolors='k')

In [ ]:
mask_core_range.plot()

In [ ]:
x

In [ ]:
file_isf.longitude.sel(x=x_min,y=y_min)

In [ ]:
seeds_pierre = pd.read_csv('/bettik/mathiotp/NEMO/INPUT_FILE/ICB/eORCA1_calving_b0.2_v0.0/eORCA025_mskisf_c3.0_v0.0.txtlatlon',header=None, sep=r'\s{2,}')

In [ ]:
seeds_pierre 

ABOVE, I SHOULD NOW GROUP INTERESTING ICE SHELVES TOGETHER

In [ ]:
summary_mask_basins

FROM HERE, IT WAS ONLY TEST WITH SEEDING

In [ ]:
file_conc_cut['isfdraft_conc'].isel(time=30).where((file_TS_cut['so'].max('deptht') > 0), 0).plot()

In [ ]:
file_mask_cut['ls_mask012'].isel(time=30).plot()

In [ ]:
file_mask_cut['ls_mask012'].isel(time=30).where((file_TS_cut['so'].max('deptht') > 0), 2).plot() # there are a few ice shelf patches inland

In [ ]:
file_TS_orig = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_2000.nc')

In [ ]:
file_TS_orig['so'].max('deptht').plot(cmap=mpl.cm.Reds_r)

In [ ]:
file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy54.nc')

In [ ]:
file_isf['ground_mask'].plot()

In [ ]:
file_isf['ISF_mask'].plot(vmin=0,vmax=10)

In [ ]:
file_isf['IF_mask'].plot(vmax=10)

In [ ]:
#file_isf['GL_mask'].where(file_isf['GL_mask'] == 66, drop=True).plot()
file_isf['IF_mask'].where(file_isf['IF_mask'] == 66, drop=True).plot()

In [ ]:
file_isf['dIF'].plot(vmax=100000)

In [ ]:
box_2D = xr.open_dataset(outputpath_boxes + 'nemo_5km_boxes_2D_oneFRIS_yy35.nc')

In [ ]:
box_2D['box_location'].sel(box_nb_tot=5).plot()

In [ ]:
file_isf_00 = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy00.nc')

In [ ]:
file_mask_54 = file_mask_cut.isel(time=54)

In [ ]:
isf_mask_default = file_mask_54['ls_mask012'].where(file_isf_00['ISF_mask'] < 3).combine_first(file_isf_00['ISF_mask']).plot(vmax=3)

In [ ]:
def def_ground_mask(file_msk, dist, add_fac):

    """
    Define a mask for the Antarctic continent as such (not the islands). 
    
    This function defines the points that are part of the Antarctic continent as such (not the islands). 
    
    Parameters
    ----------
    file_msk : xr.DataArray
        Mask separating ocean (0), ice shelves (between 0 and 2, excluding 0 and 2), grounded ice (2)
    dist : int
        Defines the size of the starting square - should be small if the resolution is coarse and high if the resolution is fine. Default is currently 150 but you can play around. A good indicator to see if it is too high is if you see the small upper tail of the Ross ice shelf or if it is masked as ground.
    add_fac : int
       Defines additional iterations. Was introduced to get to the end of the Antarctic Peninsula, sometimes it would not get there otherwise. Current default is 100 but you are welcome to play around with it.
    Returns
    -------
    mask_ground : xr.DataArray
        Array showing the coverage of the Antarctic continent (0 for islands, 1 for ocean and ice shelves, 2 for mainland)
    """  
    
    mask_10 = file_msk.where(file_msk == 2, 0).where(file_msk != 2,1) #set all ice shelves and open ocean to 0, set all grounded ice to 1

    mask_gnd = mask_10.where(mask_10>0, drop=True)
    mask_gnd = mask_gnd.where(mask_gnd>0,0)
    
    meshx_gnd, meshy_gnd = np.meshgrid(mask_gnd.x,mask_gnd.y)
    meshx_gnd_da = mask_gnd.copy(data=np.broadcast_to(meshx_gnd, mask_gnd.shape))
    meshy_gnd_da = mask_gnd.copy(data=np.broadcast_to(meshy_gnd, mask_gnd.shape))
    
    dx = abs(meshx_gnd_da.x[2] - meshx_gnd_da.x[1])
    dy = abs(meshx_gnd_da.y[2] - meshx_gnd_da.y[1])
    
    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)
    
    mask_core = mask_gnd.where(~((uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-2*dist*dy,2*dist*dy]))), 5)
    
    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1
    
    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])
    
    if 'time' in mask_gnd.dims:

        iter_list = []
        for tt,timet in enumerate(tqdm(mask_gnd.time)):

            mask_core_tt = mask_core.isel(time=tt)
            iter_mask = mask_core_tt.copy()
            for n in range(half_range+2*dist+add_fac):
                corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
                iter_mask = iter_mask.where(~((corr >= 5) & (mask_core_tt == 1)),5)

            iter_list.append(iter_mask)

        iter_mask_all = xr.concat(iter_list,dim='time')
        iter_mask_all = iter_mask_all.assign_coords({'time': mask_gnd.time})

        mask_ground = iter_mask_all.where(iter_mask_all !=5, 2).reindex_like(mask_10)
    
    else:
        
        iter_mask = mask_core.copy()
        for n in tqdm(range(half_range+2*dist+add_fac)):
            corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
            iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

        mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)
        
    mask_ground = mask_ground.where(mask_ground>0,0)
    
    return mask_ground

In [ ]:
file_isf_00['ISF_mask'].plot()

In [ ]:
FRIS_00 = file_conc_cut['isfdraft_conc'].isel(time=0).where(file_isf_00['ISF_mask'] == 11, drop=True)
FRIS_00_test = file_conc_cut['isfdraft_conc'].isel(time=0).where(file_isf_00['ISF_mask'] == 11, drop=True) * 0 + 1
FRIS_00_test = FRIS_00_test.where(np.isfinite(FRIS_00_test),0)

In [ ]:
FRIS_00_test.plot()

In [ ]:
### THIS IS THE ROUTINE THAT NEEDS TO BE IMPLEMENTED IN MASK SCRIPTS => WORKS WELL
dist = 1
dist_2 = 4
add_fac = 100
tt = 0

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

for kisf in tqdm(file_isf.Nisf.where(file_isf.Nisf >= 10, drop=True)):
    
    seed_mask = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
    seed_mask = seed_mask.where(np.isfinite(seed_mask),0)
    
    meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
    meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
    meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))
    
    if ((len(meshx_gnd_da.x) > 3) and (len(meshx_gnd_da.y) > 3)) :
        dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
        dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

        center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
        center_y = (meshx_gnd_da.y.max() + meshx_gnd_da.y.min())/2

        max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
        half_range = round(max_len_xy/2)

        mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                              & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

        if seed_mask.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
            n = 0
            while seed_mask.where(mask_core_range).max() < 1:
                #print(seed_mask.where(mask_core_range).max().values)
                center_x = meshx_gnd_da.x.min() + n*dx
                mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                        & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
                n = n+1


        mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                              & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

        # filter that checks the point around
        weights_filter = np.zeros((3,3))
        weights_filter[0,1] = 1
        weights_filter[1,0] = 1
        weights_filter[1,2] = 1
        weights_filter[2,1] = 1

        weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

        iter_mask = mask_core.copy()
        for n in range(half_range+2*dist+add_fac):
            corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
            iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

        iter_mask = iter_mask.where(iter_mask == 5)
        iter_mask = iter_mask * 0 + kisf

        kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
        seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
 meshx_gnd
    
    
    

In [ ]:
mask_33 = new_isf_mask.copy()

In [ ]:
isf_conc_tt.where(new_isf_mask < 2).plot()

In [ ]:
mask_33.plot()

In [ ]:
### THIS IS THE ROUTINE THAT NEEDS TO BE IMPLEMENTED IN MASK SCRIPTS => WORKS WELL
dist = 1
dist2 = 10
add_fac = 100
tt = 34
kisf = 23

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask_prev = mask_33
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

for kisf in tqdm(file_isf.Nisf.where(file_isf.Nisf >= 10, drop=True)):
    

    if (isf_mask_prev == kisf).sum() > 1:
        isf_domain = isf_conc_tt.where(isf_mask_prev == kisf, drop=True) * 0 + 1
    elif (isf_mask == kisf).sum() > 1:
        isf_domain = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
    isf_domain = isf_domain.where(np.isfinite(isf_domain),0)

    if kisf in [26,42,55]:

        seeded_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == kisf))

    elif ((len(isf_domain.x) > 3) and (len(isf_domain.y) > 3)):

        dx = abs(isf_domain.x[1] - isf_domain.x[0])
        dy = abs(isf_domain.y[1] - isf_domain.y[0])

        meshx_restric, meshy_restric = np.meshgrid(isf_domain.x,isf_domain.y)
        meshx_restric_da = isf_domain.copy(data=np.broadcast_to(meshx_restric, isf_domain.shape))
        meshy_restric_da = isf_domain.copy(data=np.broadcast_to(meshy_restric, isf_domain.shape))

        center_x = (meshx_restric_da.x.max() + meshx_restric_da.x.min())/2
        center_y = (meshy_restric_da.y.max() + meshy_restric_da.y.min())/2

        max_len_xy = max(len(meshx_restric_da.x),len(meshy_restric_da.y))
        half_range = round(max_len_xy/2)

        mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                              & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

        if kisf == 53:
            center_y = isf_domain.y.max()

        elif isf_domain.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
            n = 0
            while isf_domain.where(mask_core_range).max() < 1:
                #print(center_x)
                #print(seed_mask.where(mask_core_range).max().values)
                center_x = isf_domain.x.min() + n*dx
                mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                        & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
                n = n+1

        # enlarge the domain
        if isf_domain.x[0] < isf_domain.x[1]:
            xxrange = np.arange(isf_domain.x.min() - dist2*dx, isf_domain.x.max() + dist2*dx,dx)
        else:
            xxrange = np.arange(isf_domain.x.max() + dist2*dx, isf_domain.x.min() - dist2*dx,-dx)

        if isf_domain.y[0] < isf_domain.y[1]:
            yyrange = np.arange(isf_domain.y.min() - dist2*dy, isf_domain.y.max() + dist2*dy,dy)
        else:
            yyrange = np.arange(isf_domain.y.max() + dist2*dy, isf_domain.y.min() - dist2*dy,-dy)

        new_domain = isf_conc_tt.sel(x=xxrange,y=yyrange) 
        seed_mask = isf_conc_tt.where(new_domain, drop=True) * 0 + 1
        seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

        meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
        meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
        meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))


        mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                              & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

        # filter that checks the point around
        weights_filter = np.zeros((3,3))
        weights_filter[0,1] = 1
        weights_filter[1,0] = 1
        weights_filter[1,2] = 1
        weights_filter[2,1] = 1

        weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

        iter_mask = mask_core.copy()
        for n in range(half_range+2*dist+add_fac):
            corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
            iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

        iter_mask = iter_mask.where(iter_mask == 5)
        iter_mask = iter_mask * 0 + kisf

        kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
        seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
new_isf_mask.where(new_isf_mask == 23).plot()

In [ ]:
isf_dim_orig = np.array([10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75])

In [ ]:
isf_conc_tt.where(new_isf_mask==10).plot()

In [ ]:
for kisf in isf_dim_orig[20:40]:
    plt.figure()
    isf_conc_tt.where(new_isf_mask==kisf).plot()
    plt.title(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
dist = 1
dist_2 = 4
add_fac = 100
tt = 0

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

In [ ]:
isf_mask.plot()

In [ ]:
file_isf['isf_name'].where(file_isf['isf_name']=='Larsen B',drop=True)

In [ ]:
isf_mask.where(isf_mask<2).plot()

In [ ]:
isf_domain = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
isf_domain = isf_domain.where(np.isfinite(isf_domain),0)

if ((len(isf_domain.x) > 3) and (len(isf_domain.y) > 3)) :

    dx = abs(isf_domain.x[1] - isf_domain.x[0])
    dy = abs(isf_domain.y[1] - isf_domain.y[0])

    meshx_restric, meshy_restric = np.meshgrid(isf_domain.x,isf_domain.y)
    meshx_restric_da = isf_domain.copy(data=np.broadcast_to(meshx_restric, isf_domain.shape))
    meshy_restric_da = isf_domain.copy(data=np.broadcast_to(meshy_restric, isf_domain.shape))

    center_x = (meshx_restric_da.x.max() + meshx_restric_da.x.min())/2
    center_y = (meshy_restric_da.y.max() + meshy_restric_da.y.min())/2

    max_len_xy = max(len(meshx_restric_da.x),len(meshy_restric_da.y))
    half_range = round(max_len_xy/2)

    mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
    if kisf == 53:
        center_y = isf_domain.y.max()
        
    elif isf_domain.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
        n = 0
        while isf_domain.where(mask_core_range).max() < 1:
            #print(center_x)
            #print(seed_mask.where(mask_core_range).max().values)
            center_x = isf_domain.x.min() + n*dx
            mask_core_range = ((uf.in_range(meshx_restric_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                    & uf.in_range(meshy_restric_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
            n = n+1

    # enlarge the domain
    if isf_domain.x[0] < isf_domain.x[1]:
        xxrange = np.arange(isf_domain.x.min() - dist2*dx, isf_domain.x.max() + dist2*dx,dx)
    else:
        xxrange = np.arange(isf_domain.x.max() + dist2*dx, isf_domain.x.min() - dist2*dx,-dx)

    if isf_domain.y[0] < isf_domain.y[1]:
        yyrange = np.arange(isf_domain.y.min() - dist2*dy, isf_domain.y.max() + dist2*dy,dy)
    else:
        yyrange = np.arange(isf_domain.y.max() + dist2*dy, isf_domain.y.min() - dist2*dy,-dy)

    new_domain = isf_conc_tt.sel(x=xxrange,y=yyrange) 
    seed_mask = isf_conc_tt.where(new_domain, drop=True) * 0 + 1
    seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

    meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
    meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
    meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))


    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)


    



In [ ]:
mask_core.plot()

In [ ]:
mask_33.plot()

In [ ]:
iter_mask.plot()

In [ ]:
file_isf['isf_name'].sel(Nisf=53)

In [ ]:
iter_mask.plot()

In [ ]:
isf_conc_tt.where(file_isf['ISF_mask']==53).plot()

In [ ]:
new_isf_mask.plot()#.plot(vmin=25,vmax=27)

In [ ]:
isf_conc_tt.where(new_isf_mask < 1).plot()

In [ ]:
file_isf['Nisf'].where(file_isf['isf_name'] == 'Cosgrove')

In [ ]:
meshy_restric_da.plot()

In [ ]:
iter_mask.plot()

In [ ]:
seed_mask.plot()

In [ ]:
mask_core_range.plot()

In [ ]:
isf_domain.plot()

In [ ]:
meshx_restric_da.plot()

In [ ]:
mask_core.plot()

In [ ]:
isf_domain.x.min() + n*dx

In [ ]:
mask_33.plot()

In [ ]:
new_isf_mask.plot(vmin=24,vmax=26)

In [ ]:
file_isf['isf_name'].sel(Nisf=25)

In [ ]:
isf_conc_tt.where(new_isf_mask==0).plot()

In [ ]:
Iceshelf['ID_isf']

In [ ]:
summary_mask_basins = xr.open_dataset('/bettik/burgardc/DATA/NN_PARAM/interim/basins_mask_extrap_50km.nc')

In [ ]:
file_msk = file_mask_cut['ls_mask012']
isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))
isf_mask_all = summary_mask_basins['ID_isf'].where(np.isfinite(isf_only_mask))

In [ ]:
file_mask_cut['ls_mask012'].isel(time=30).plot()

In [ ]:
file_conc_cut['isfdraft_conc'].where(np.isnan(isf_mask_all)).isel(time=0).plot()

In [ ]:
isf_mask_all.isel(time=34).plot()

In [ ]:
isf_only_mask = file_msk.where((file_msk > 0) & (file_msk < 2))

In [ ]:
isf_only_mask.plot()

In [ ]:
isf_mask_all.plot()

In [ ]:
file_msk_NEMOgrid = xr.open_dataset(inputpath_data+'custom_lsmask_Ant.nc')

In [ ]:
NEMO_msk = file_msk_NEMOgrid['ls_mask012'].isel(time=0)

In [ ]:
NEMO_msk.plot()

In [ ]:
file_msk.plot()

In [ ]:
isf_mask_all.where(isf_mask_all == np.array([1,2,3]))

In [ ]:
file_TS_cut['so'].max('deptht').plot()

In [ ]:
summary_mask_basins['name_isf']

In [ ]:
drainage_basins.x[1] - drainage_basins.x[0]

In [ ]:
Iceshelf['ID_isf'].min()

In [ ]:
Iceshelf['ID_isf'].where(isf_conc_tt>0).plot()

In [ ]:
isf_conc_tt.where(np.isnan(drainage_basins)).plot()

In [ ]:
isf_domain.x.max()

In [ ]:
isf_domain.x.min()

In [ ]:
xxrange

In [ ]:
seed_mask.x

In [ ]:
plt.figure()
isf_mask_prev.where(isf_mask_prev == 38).plot()

In [ ]:
seed_mask = isf_conc_tt.sel(x=xxrange,y=yyrange).plot()

In [ ]:
iter_mask.plot()

In [ ]:
seed_mask.plot()

In [ ]:
xxrange

In [ ]:
mask_33.plot(vmin=74,vmax=76)

In [ ]:
file_isf['ISF_mask'].where(file_isf['ISF_mask']==80).plot()

In [ ]:
mask_33 = new_isf_mask.copy()

In [ ]:
isf_conc_tt.where(new_isf_mask<2).plot()

In [ ]:
seed_mask = isf_conc_tt.where(isf_mask == 10, drop=True) * 0 + 1

In [ ]:
file_isf['ISF_mask'].where(file_isf['ISF_mask']==75).plot()

In [ ]:
new_isf_mask.plot()

In [ ]:
plt.figure()
new_isf_mask.where(new_isf_mask==10).plot()

In [ ]:
(new_isf_mask - file_isf['ISF_mask']).plot()

In [ ]:
plt.figure()
file_isf['ISF_mask'].plot()

In [ ]:
file_isf['isf_name'].sel(Nisf=23)

In [ ]:
center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
    
mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))

In [ ]:
seed_mask.plot()

In [ ]:
### THIS IS TO TEST FOR REMOVING
dist = 1
add_fac = 100
tt = 30

file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy'+str(tt).zfill(2)+'.nc').drop('time')
isf_conc_tt = file_conc_cut['isfdraft_conc'].sel(time=tt)
isf_mask = file_isf['ISF_mask']
ground_mask = file_isf['ground_mask']
seeded_isf_mask = isf_mask.copy() * np.nan

kisf = 25
seed_mask = isf_conc_tt.where(isf_mask == kisf, drop=True) * 0 + 1
seed_mask = seed_mask.where(np.isfinite(seed_mask),0)

meshx_gnd, meshy_gnd = np.meshgrid(seed_mask.x,seed_mask.y)
meshx_gnd_da = seed_mask.copy(data=np.broadcast_to(meshx_gnd, seed_mask.shape))
meshy_gnd_da = seed_mask.copy(data=np.broadcast_to(meshy_gnd, seed_mask.shape))

if ((len(meshx_gnd_da.x) > 3) and (len(meshx_gnd_da.y) > 3)) :
    dx = abs(meshx_gnd_da.x[1] - meshx_gnd_da.x[0])
    dy = abs(meshx_gnd_da.y[1] - meshx_gnd_da.y[0])

    center_x = (meshx_gnd_da.x.max() + meshx_gnd_da.x.min())/2
    center_y = (meshx_gnd_da.y.max() + meshx_gnd_da.y.min())/2

    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)

#    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
#                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)
    
    mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
    
    if seed_mask.where(mask_core_range).max() < 1: # if the ice shelf is not in the middle of the cell
        n = 0
        while seed_mask.where(mask_core_range).max() < 1:
            print(seed_mask.where(mask_core_range).max().values)
            center_x = meshx_gnd_da.x.min() + n*dx
            mask_core_range = ((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) 
                    & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))
            n = n+1

        
#    if seed_mask.sel(x=center_x,y=center_y) < 2:
#        center_x = 0
#    while seed_mask.sel(x=center_x+dx,y=center_y+dy) < 2:
#          center_x = center_x + 1
    mask_core = seed_mask.where(~((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy]))), 5)

    # filter that checks the point around
    weights_filter = np.zeros((3,3))
    weights_filter[0,1] = 1
    weights_filter[1,0] = 1
    weights_filter[1,2] = 1
    weights_filter[2,1] = 1

    weights_da = xr.DataArray(data=weights_filter,dims=['y0','x0'])

    iter_mask = mask_core.copy()
    for n in range(half_range+2*dist+add_fac):
        corr = pf.xr_nd_corr_v2(iter_mask, weights_filter)
        iter_mask = iter_mask.where(~((corr >= 5) & (mask_core == 1)),5)

    iter_mask = iter_mask.where(iter_mask == 5)
    iter_mask = iter_mask * 0 + kisf

    #kisf_reg_grid = iter_mask.reindex_like(isf_mask)    
    #seeded_isf_mask = seeded_isf_mask.combine_first(kisf_reg_grid)

#new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 0))
#new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)
#mask_ground = iter_mask.where(iter_mask !=5, 2).reindex_like(mask_10)

#mask_ground = mask_ground.where(mask_ground>0,0)

In [ ]:
seed_mask.where(((uf.in_range(meshx_gnd_da, [-1*dist*dx + center_x,center_x + 1*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-1*dist*dy + center_y,center_y +1*dist*dy])))).plot()

In [ ]:
plt.figure()
seed_mask.plot()

In [ ]:
plt.figure()
mask_core.plot()

In [ ]:
iter_mask.plot()

In [ ]:
ls_mask = file_mask_cut['ls_mask012'].sel(time=tt)

In [ ]:
new_isf_mask = seeded_isf_mask.combine_first(isf_mask.where(isf_mask == 1))
new_isf_mask = new_isf_mask.where(new_isf_mask > 0,0)

In [ ]:
new_isf_mask.plot()

In [ ]:
ls_mask.where(file_TS_cut['so'].max() > 0).plot()

In [ ]:
plt.figure()
iter_mask.plot()

In [ ]:
iter_mask.plot()

In [ ]:
plt.figure()
FRIS_00_test.plot()

In [ ]:
FRIS_00.where(np.isfinite(FRIS_00), 0).plot()

In [ ]:
iter_mask.where(iter_mask == 5).plot()

In [ ]:
(((corr >= 5) & (mask_core == 1)) ^ (corr >= 5)).plot()

In [ ]:
iter_mask.where(iter_mask !=5, 2).plot()

In [ ]:
corr.plot()

In [ ]:
iter_mask.

In [ ]:
iter_mask.where((corr >= 5)).plot()

In [ ]:
uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]).plot()

In [ ]:
meshx_gnd_da.plot()

In [ ]:
meshx_gnd_da.plot()

In [ ]:
uf.in_range(meshx_gnd_da, [-dx,dx]).plot()

In [ ]:
    dx = abs(meshx_gnd_da.x[2] - meshx_gnd_da.x[1])
    dy = abs(meshx_gnd_da.y[2] - meshx_gnd_da.y[1])
    
    max_len_xy = max(len(meshx_gnd_da.x),len(meshx_gnd_da.y))
    half_range = round(max_len_xy/2)
    
    mask_core = mask_gnd.where(~((uf.in_range(meshx_gnd_da, [-2*dist*dx,2*dist*dx]) # assuming that the South Pole is in the center of the projection
                          & uf.in_range(meshy_gnd_da, [-2*dist*dy,2*dist*dy]))), 5)